## Install CatBoost

In [ ]:
pip install catboost

## Import Dependencies

In [ ]:
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.metrics import (mean_squared_error,
                             explained_variance_score,
                             max_error,
                             mean_absolute_error,
                             median_absolute_error,
                             r2_score)

## Format Pandas and Read CSV Data

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 999)
url = 'https://gist.githubusercontent.com/simon-benigeri/d609a89f0d962e47f728b937471c12cc/raw/c1465fc30bc0f3d1cf9bc975deb987ea884df685/checkpoint_4_data'
df = pd.read_csv(url)

## Format Data

In [ ]:
df.head()

,officer_id,year,salary,race,gender,birth_year,appointed_year,trr_count,hm_count,allegations_count,sustained_count
0,1,2007,58896,White,M,1971.0,2005.0,0,4,1,0
1,1,2008,61932,White,M,1971.0,2005.0,0,8,0,0
2,1,2009,64992,White,M,1971.0,2005.0,1,11,0,0
3,1,2010,73164,White,M,1971.0,2005.0,1,11,1,0
4,1,2011,74628,White,M,1971.0,2005.0,1,13,0,0


**Compute Experience and Age**


*   experience = year - appointed year
*   age = year - birth year
*   remove rows where exp < 0 and age <= 0





In [ ]:
df['experience'] = pd.to_numeric(df.apply(lambda row: row['year'] - row['appointed_year'], axis=1), errors='coerce', downcast='integer')
df['age'] = pd.to_numeric(df.apply(lambda row: row['year'] - row['birth_year'], axis=1), errors='coerce', downcast='integer')
df['year'] = pd.to_datetime(df['year'], format='%Y')

df = df[df['experience']>=0]
df = df[df['age']>0]

**Compute Next Year's Salary and % Raise**

*   Next Year's Salary = Salary for year + 1
*   remove rows where there is no next year's salary, so for example, year=2015

*   % Raise = 100 * 1 - salary/next year's salary
*   remove rows where raise  < 0 and

In [ ]:
df['next_salary'] = df.sort_values(by=['year'], ascending=True).groupby(['officer_id'])['salary'].shift(-1)
df = df[df['next_salary'].notnull()]

df['raise'] = df.apply(lambda row: 100*(1. - row['salary']/row['next_salary']), axis=1)
df = df[df['raise']>=0]

**Clean up the dataframe**

*   drop 'birth_year', 'appointed_year', 'next_salary'

In [ ]:
df.drop(columns=['birth_year', 'appointed_year', 'next_salary'], inplace=True)
df.sort_values(by=['year'], ascending=True, inplace=True)

## DATA EXPLORATION ##

**We use the next few cells to show you some elements we thought were significant**

**1. Correlation Analysis**

*   As expected, strong positive correlations between experience, age, and salary
*   Positive correlation between raise and year
*   Strong negative correlation between salary and year
*   Negative correlation between raise and experience and age
*   TRRs, allegations, honorable mentions, sustained allegations seem to have little to no correlation with raise

In [ ]:
df.corr()

,officer_id,salary,trr_count,hm_count,allegations_count,sustained_count,experience,age,raise
officer_id,1.000000,-0.002583,0.028239,0.030924,0.038289,-0.000480,-0.025091,-0.038374,-0.010735
salary,-0.002583,1.000000,-0.103111,-0.116256,-0.105376,-0.006115,0.734985,0.594448,-0.627751
trr_count,0.028239,-0.103111,1.000000,0.353181,0.352012,0.042530,-0.261296,-0.279220,-0.010721
hm_count,0.030924,-0.116256,0.353181,1.000000,0.355241,0.006597,-0.267680,-0.305088,-0.010970
allegations_count,0.038289,-0.105376,0.352012,0.355241,1.000000,0.230412,-0.187117,-0.195534,-0.045327
sustained_count,-0.000480,-0.006115,0.042530,0.006597,0.230412,1.000000,-0.011898,0.000077,-0.014986
experience,-0.025091,0.734985,-0.261296,-0.267680,-0.187117,-0.011898,1.000000,0.814890,-0.344328
age,-0.038374,0.594448,-0.279220,-0.305088,-0.195534,0.000077,0.814890,1.000000,-0.284793
raise,-0.010735,-0.627751,-0.010721,-0.010970,-0.045327,-0.014986,-0.344328,-0.284793,1.000000


**2. As suspected, the raises vary with the years.**

- Note that 2013 and 2009 have high mean raises

In [ ]:
raises = pd.DataFrame(zip(list(df['year'].unique()), 
                          list(df.groupby(['year'])['raise'].mean()), 
                          list(df.groupby(['year'])['raise'].median()),
                          list(df.groupby(['year'])['raise'].std()),
                          list(df.groupby(['year'])['raise'].var()),
                          list(df.groupby(['year'])['raise'].skew())),
                      columns=['year', 'mean raise', 'median raise', 'std raise', 'var raise', 'skew raise'])
raises

,year,mean raise,median raise,std raise,var raise,skew raise
0,2007-01-01,2.264243,0.000000,5.183327,26.866884,3.570471
1,2008-01-01,1.610521,0.000000,3.340903,11.161635,4.741325
2,2009-01-01,8.115766,6.700016,3.028777,9.173489,4.751789
3,2010-01-01,3.517023,1.961730,3.987775,15.902352,5.413427
4,2011-01-01,1.908440,0.989202,2.529715,6.399456,7.040216
5,2012-01-01,1.978371,0.000000,6.218376,38.668205,4.046976
6,2013-01-01,8.261752,5.768952,7.192851,51.737108,3.347655
7,2014-01-01,3.123902,0.991678,5.527625,30.554638,4.475517


**3. We look at experience and raises here**

*   New recruits are quite significant. Academy graduates get the highest raises.
*   Note that 2013 and 2009 have abnormally high mean raises. 2013 has a surge of new recruits. 2009 does not.

In [ ]:
exp = df.pivot(index='officer_id', columns='experience', values='raise')
exp.describe()

experience,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0
count,2158.000000,2467.000000,2374.000000,2404.000000,2751.000000,3044.000000,3401.000000,3854.000000,4198.000000,4150.000000,4042.000000,3971.000000,4115.000000,4196.000000,3892.000000,3578.000000,3433.000000,3141.000000,2830.000000,2564.000000,2123.000000,1970.000000,1840.000000,1538.000000,1091.000000,950.000000,828.000000,744.000000,542.000000,279.000000,202.000000,191.000000,140.000000,94.000000,101.000000,89.000000,88.000000,83.000000,59.000000,43.000000,25.000000,10.000000,4.000000
mean,29.945839,7.557644,6.174837,7.194512,5.145563,1.785657,1.819387,2.198197,1.915049,5.099972,2.427892,2.300677,1.862023,1.791803,5.490019,2.681124,1.739173,1.655250,2.745771,5.670569,1.777306,1.409182,2.241755,3.041569,4.531525,1.535846,1.241971,2.888888,2.169318,2.478409,0.621375,1.702142,2.886024,3.458993,0.935200,0.155371,3.067677,1.613502,1.908349,2.658793,1.550149,3.546425,0.978926
std,6.254785,4.444405,2.307352,2.823751,3.431052,2.416512,2.569398,2.678729,2.519403,2.471780,2.760766,2.623814,2.371794,2.475391,2.808367,2.734471,2.223415,2.336102,2.993731,2.766277,2.468623,2.297755,2.700611,2.746097,2.231547,1.786891,2.206925,2.930710,2.287522,2.694119,1.158039,2.697156,3.150586,2.710516,1.524172,0.390606,3.290003,2.432848,2.659767,2.931722,2.104305,3.340102,1.130366
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.828009,5.378973,4.902151,4.708272,1.961730,0.000000,0.000000,0.000000,0.000000,3.388247,0.000000,0.000000,0.000000,0.000000,3.364517,0.988554,0.000000,0.000000,0.000000,4.515805,0.000000,0.000000,0.000000,0.989176,3.775688,0.000000,0.000000,0.000000,0.987911,0.987911,0.000000,0.000000,0.000000,0.989408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.247358,0.000000
50%,29.946368,5.869352,4.902151,5.747492,4.790367,0.987104,0.987104,0.987104,0.987104,4.337794,0.992155,0.992155,0.992155,0.990241,4.318418,0.991678,0.990688,0.988554,0.991678,4.552400,0.989558,0.000000,0.989176,1.957541,3.776160,0.989202,0.000000,0.987911,0.989202,0.989202,0.000000,0.000000,0.987911,1.962992,0.000000,0.000000,0.000000,0.000000,0.989433,1.957852,0.989433,3.841211,0.978926
75%,33.982724,10.818332,5.869352,11.078567,6.673099,1.961730,1.961730,5.761440,1.961730,5.274606,5.768952,5.768952,1.957586,1.957586,8.930448,5.764516,1.959312,1.959312,5.764516,9.125296,1.961920,0.989768,5.768322,6.690297,4.721030,1.962992,0.989202,5.769988,1.962992,6.690333,0.989202,0.989951,6.691713,6.695852,0.989433,0.000000,6.698100,1.957852,1.959516,6.688654,1.959516,6.698100,1.957852
max,37.525002,37.525002,11.271807,15.345762,29.946368,21.495869,29.946368,29.946368,26.521240,21.140788,22.321900,18.379357,17.575874,26.813366,24.686621,35.523810,18.504648,11.128938,49.740163,29.946368,15.109718,11.860805,6.704444,8.520959,10.013292,8.464202,12.080278,12.080278,6.698100,9.315344,5.769988,6.698100,10.958154,9.315344,6.698100,1.957852,6.698100,6.698100,6.698100,6.698100,6.698100,6.698100,1.957852


In [ ]:
rookies = df[df['experience']<1]
rookies.count()

officer_id           2158
year                 2158
salary               2158
race                 2158
gender               2158
trr_count            2158
hm_count             2158
allegations_count    2158
sustained_count      2158
experience           2158
age                  2158
raise                2158
dtype: int64

In [ ]:
rookies = df[df['experience']==0].groupby('year').count()
rookies['officer_id']

year
2007-01-01    373
2008-01-01    119
2009-01-01    133
2010-01-01    171
2011-01-01     49
2012-01-01    436
2013-01-01    573
2014-01-01    304
Name: officer_id, dtype: int64

## Model Training

### Train / Validation / Test Set Split
### 0.64 / 0.16 / 0.20

- **Train:** X_train, y_train
- **Val:** X_val, y_val
- **Test:** X_test, y_test
- **TRAIN:** X_train+X_val, y_train+y_val denoted X_TRAIN, y_TRAIN

In [ ]:
features = df.drop(columns=['officer_id', 'raise'])
targets = df['raise']
X_TRAIN, X_test, y_TRAIN, y_test = train_test_split(features, targets, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_TRAIN, y_TRAIN, test_size=0.2)

### Model Creation
#### Instantiate the model

**Note:**

*   We use hyperparameters that were determined using gridsearch and 3-fold cross validation, optimizing Root Mean Squared Error
*   We pass 'race' and 'gender' as categorical features. No need for one-hot encoding with catboost



In [ ]:
# regressor = CatBoostRegressor(cat_features=('race', 'gender'), loss_function='RMSE', verbose=0)# , depth=6, learning_rate=0.1, l2_leaf_reg=1)
regressor = CatBoostRegressor(cat_features=('race', 'gender'), loss_function='RMSE', verbose=0, depth=6, learning_rate=0.1, l2_leaf_reg=1)

#### Used hyperparameters

In [ ]:
regressor.get_params()

{'cat_features': ('race', 'gender'),
 'depth': 6,
 'l2_leaf_reg': 1,
 'learning_rate': 0.1,
 'loss_function': 'RMSE',
 'verbose': 0}

#### Fitting the model

**We fit on the training set**

In [ ]:
regressor.fit(X_train, y_train)

## Evaluation

#### Predict

In [ ]:
yhat_train = regressor.predict(X_train)

In [ ]:
yhat_val = regressor.predict(X_val)

#### Metrics

*   RMSE (Root Mean Squared Error)
*   Explained Variance
*   Max Error
*   Mean Absolute Error
*   Median Absolute Error

*   R^2 (coefficient of determination) regression score function. Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse. A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.


In [ ]:
def compute_metrics(y_hat, y):
  """
  y_hat: predicted values
  y: expected values
  """
  rmse = mean_squared_error(y, y_hat, squared=False)
  explained_variance = explained_variance_score(y, y_hat)
  max_error_ = max_error(y, y_hat)
  mae = mean_absolute_error(y, y_hat)
  median_abs_error_ = median_absolute_error(y, y_hat)
  r2 = r2_score(y, y_hat)
  df_metrics = pd.DataFrame(data=np.asarray([rmse, explained_variance, max_error_, mae, median_abs_error_, r2], 
                                            dtype=float).reshape(1, -1),
                            columns=('Root Mean Squared Error', 'Explained Variance', 'Max Error', 
                                     'Mean Absolute Error', 'Median Absolute Error', 'R2 Score'))
  return df_metrics

In [ ]:
evaluation_train = compute_metrics(yhat_train, y_train)
evaluation_val = compute_metrics(yhat_val, y_val)

In [ ]:
evaluation_report = pd.concat([evaluation_train, evaluation_val], keys=['train', 'validation'], axis=0)
evaluation_report

,,Root Mean Squared Error,Explained Variance,Max Error,Mean Absolute Error,Median Absolute Error,R2 Score
train,0,0.928437,0.971485,42.607566,0.205483,0.055777,0.971485
validation,0,1.055219,0.963735,36.513405,0.221088,0.056779,0.963735


#### Fitting the model on the full Training set

**We fit on the full training set**

In [ ]:
# final_regressor = CatBoostRegressor(cat_features=('race', 'gender'), loss_function='RMSE', verbose=0) #depth=6, learning_rate=0.1, l2_leaf_reg=1)
# 'depth': 8, 'l2_leaf_reg': 0.5, 'learning_rate': 0.1
final_regressor = CatBoostRegressor(cat_features=('race', 'gender'), loss_function='RMSE', verbose=0, depth=6, learning_rate=0.1, l2_leaf_reg=1)
#depth=6, learning_rate=0.1, l2_leaf_reg=1)

In [ ]:
final_regressor.fit(X_TRAIN, y_TRAIN)

In [ ]:
yhat_test = final_regressor.predict(X_test)

In [ ]:
evaluation_test = compute_metrics(yhat_test, y_test)

In [ ]:
evaluation_report = pd.concat([evaluation_train, evaluation_val, evaluation_test], keys=['train', 'validation', 'test'], axis=0)
evaluation_report

,,Root Mean Squared Error,Explained Variance,Max Error,Mean Absolute Error,Median Absolute Error,R2 Score
train,0,0.928437,0.971485,42.607566,0.205483,0.055777,0.971485
validation,0,1.055219,0.963735,36.513405,0.221088,0.056779,0.963735
test,0,1.016656,0.965122,30.622175,0.214826,0.051995,0.965121


### Feature Importance

In [ ]:
pd.DataFrame(data=final_regressor.get_feature_importance().reshape(1, -1), columns=final_regressor.feature_names_)

,year,salary,race,gender,trr_count,hm_count,allegations_count,sustained_count,experience,age
0,40.711721,46.220302,0.281906,0.171524,0.015639,0.394493,0.059178,0.009121,11.577226,0.55889
